In [1]:
import requests
import pandas as pd
import geojson
from shapely.geometry import shape
from shapely.geometry import Point
import json as js

from dotenv import load_dotenv
import os

Here we use a .env file to ensure that the API key does not go public  
However, the API is free to use, you just have to apply [here](https://info.nobil.no/api) to get  
the key. If you don't want to apply there is also a client test of the  
api [here](https://www.nobil.no/api/client/search_apiVer3.php)

In [2]:
# Collect the api key
load_dotenv()

API_KEY = os.environ.get('API_KEY')

In [3]:
def get_api_data(ne,sw,API_KEY):
    url = 'https://nobil.no/api/server/search.php?mode=ajax'
    post_data = { 'apikey': f'{API_KEY}', 'apiversion': '3',
    'action': "search",
    'type': 'rectangle',
    'northeast': f'{ne}',
    'southwest': f'{sw}'}
    return requests.post(url, data=post_data)

def json_to_file(file,filename):
    with open(f"{filename}.json", "w") as write_file:
        js.dump(file, write_file)

In [4]:
# Ne stands for the coordinates to the northeast most point where you want to search
# Sw stands for the southwestern most point you want to search.
# Together they form a square where you ask the API to send back data
ne = 59.943921193288915, 10.826683044433594
sw = 59.883683240905256, 10.650901794433594

r = get_api_data(ne,sw,API_KEY)
jsonFile = r.json()
json_to_file(jsonFile,'example_data')